In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


seed = 42

train_df = pd.read_csv("../dataset/APPA-REAL/appa-real-release/gt_avg_train.csv")
val_df = pd.read_csv("../dataset/APPA-REAL/appa-real-release/gt_avg_valid.csv")

train_df['apparent_age_avg'] = train_df['apparent_age_avg'].apply(lambda x: round(x))
val_df['apparent_age_avg'] = val_df['apparent_age_avg'].apply(lambda x: round(x))

train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="D:/imdb/Align.5/Train/",
    x_col="file_name",
    y_col='apparent_age_avg',
    batch_size=64,
    target_size=(224, 224),
    class_mode="raw",
    shuffle=True,
    seed=seed
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="D:/imdb/Align.5/Valid/",
    x_col="file_name",
    y_col='apparent_age_avg',
    batch_size=64,
    target_size=(224, 224),
    class_mode="raw",
    shuffle=True,
    seed=seed
)

Found 4113 validated image filenames.
Found 1500 validated image filenames.


In [4]:
from keras.models import load_model
import coral_ordinal as coral
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


early_stopping = EarlyStopping(
                                patience=10,
                                min_delta = 0.01, 
                                verbose=1, 
                                mode = 'min',
                                monitor='val_loss')

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=5,
                                    episilon= 0.01,
                                    factor=0.1,
                                    cooldown = 0, 
                                    verbose=1)

model = load_model("D:/models/FINAL/Mobile/Mobile_IMDB.h5")
for layer in model.layers:
    layer.trainable=True

In [6]:
from keras import optimizers


model.compile(optimizers.Adam(learning_rate = 0.001),
              loss = coral.OrdinalCrossEntropy(num_classes = 101),
              metrics = [coral.MeanAbsoluteErrorLabels()])

score = model.fit(train_generator, epochs=200, validation_data=val_generator,verbose=True, callbacks=[
    reduce_learning_rate,
    early_stopping,
])

Epoch 1/100
65/65 [==============================] - 29s 363ms/step - loss: 16.3113 - mean_absolute_error_labels: 6.3567 - val_loss: 14.1874 - val_mean_absolute_error_labels: 5.6931 - lr: 0.0010
Epoch 2/100
65/65 [==============================] - 26s 390ms/step - loss: 12.4713 - mean_absolute_error_labels: 5.1499 - val_loss: 11.6905 - val_mean_absolute_error_labels: 4.8481 - lr: 0.0010
Epoch 3/100
65/65 [==============================] - 25s 388ms/step - loss: 11.4373 - mean_absolute_error_labels: 4.7439 - val_loss: 11.9625 - val_mean_absolute_error_labels: 4.9988 - lr: 0.0010
Epoch 4/100
65/65 [==============================] - 25s 393ms/step - loss: 10.8687 - mean_absolute_error_labels: 4.4878 - val_loss: 11.3398 - val_mean_absolute_error_labels: 4.7108 - lr: 0.0010
Epoch 5/100
65/65 [==============================] - 26s 389ms/step - loss: 10.2333 - mean_absolute_error_labels: 4.2798 - val_loss: 11.3963 - val_mean_absolute_error_labels: 4.7361 - lr: 0.0010
Epoch 6/100
65/65 [======

In [7]:
model.save("D:/models/FINAL/Mobile/Extra/Mob_balanced_weights.h5")

In [8]:
model_train_loss = score.history["loss"]
model_val_loss = score.history["val_loss"]
model_train_acc = score.history["mean_absolute_error_labels"]
model_val_acc = score.history["val_mean_absolute_error_labels"]

results = {'train-loss': model_train_loss,
        'val-loss': model_val_loss,
        'train-mae': model_train_acc,
        'val-mae': model_val_acc}
resultsdf = pd.DataFrame(results)
resultsdf.to_csv('D:/models/FINAL/Mobile/Extra/Mob_balanced_weights.csv', index=False)